<a href="https://colab.research.google.com/github/TabethaMcRee/TabethaMcRee_DTSC3020_Fall2025/blob/main/Assignment_6_WebScraping_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 6 (4 points) — Web Scraping

In this assignment you will complete **two questions**. The **deadline is posted on Canvas**.


## Assignment Guide (Read Me First)

- This notebook provides an **Install Required Libraries** cell and a **Common Imports & Polite Headers** cell. Run them first.
- Each question includes a **skeleton**. The skeleton is **not** a solution; it is a lightweight scaffold you may reuse.
- Under each skeleton you will find a **“Write your answer here”** code cell. Implement your scraping, cleaning, and saving logic there.
- When your code is complete, run the **Runner** cell to print a Top‑15 preview and save the CSV.
- Expected outputs:
  - **Q1:** `data_q1.csv` + Top‑15 sorted by the specified numeric column.
  - **Q2:** `data_q2.csv` + Top‑15 sorted by `points`.


In [2]:
#Install Required Libraries
!pip -q install requests beautifulsoup4 lxml pandas
print("Dependencies installed.")

Dependencies installed.


### 2) Common Imports & Polite Headers

In [3]:
# Common Imports & Polite Headers
import re, sys, pandas as pd, requests
from bs4 import BeautifulSoup
HEADERS = {"User-Agent": (
    "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 "
    "(KHTML, like Gecko) Chrome/122.0 Safari/537.36")}
def fetch_html(url: str, timeout: int = 20) -> str:
    r = requests.get(url, headers=HEADERS, timeout=timeout)
    r.raise_for_status()
    return r.text
def flatten_headers(df: pd.DataFrame) -> pd.DataFrame:
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = [" ".join([str(x) for x in tup if str(x)!="nan"]).strip()
                      for tup in df.columns.values]
    else:
        df.columns = [str(c).strip() for c in df.columns]
    return df
print("Common helpers loaded.")


Common helpers loaded.


## Question 1 — IBAN Country Codes (table)
**URL:** https://www.iban.com/country-codes  
**Extract at least:** `Country`, `Alpha-2`, `Alpha-3`, `Numeric` (≥4 cols; you may add more)  
**Clean:** trim spaces; `Alpha-2/Alpha-3` → **UPPERCASE**; `Numeric` → **int** (nullable OK)  
**Output:** write **`data_q1.csv`** and **print a Top-15** sorted by `Numeric` (desc, no charts)  
**Deliverables:** notebook + `data_q1.csv` + short `README.md` (URL, steps, 1 limitation)

**Tip:** You can use `pandas.read_html(html)` to read tables and then pick one with ≥3 columns.


In [5]:
# --- Q1 Skeleton (fill the TODOs) ---
def q1_read_table(html: str) -> pd.DataFrame:
    """Return the first table with >= 3 columns from the HTML.
    TODO: implement with pd.read_html(html), pick a reasonable table, then flatten headers.
    """
    raise NotImplementedError("TODO: implement q1_read_table")

def q1_clean(df: pd.DataFrame) -> pd.DataFrame:
    """Clean columns: strip, UPPER Alpha-2/Alpha-3, cast Numeric to int (nullable), drop invalids.
    TODO: implement cleaning steps.
    """
    raise NotImplementedError("TODO: implement q1_clean")

def q1_sort_top(df: pd.DataFrame, top: int = 15) -> pd.DataFrame:
    """Sort descending by Numeric and return Top-N.
    TODO: implement.
    """
    raise NotImplementedError("TODO: implement q1_sort_top")


In [13]:
# Q1 — Write your answer here
import pandas as pd

def q1_read_table(html: str) -> pd.DataFrame:
    """Return the first table with >= 3 columns from the HTML."""
    tables = pd.read_html(html)
    # Select the table with 4 columns and expected headers
    expected_headers = ["Country", "Alpha-2 code", "Alpha-3 code", "Numeric"]
    for i, t in enumerate(tables):
        df = t.copy()
        # Flatten columns first to check for expected headers
        df.columns = [''.join(col).strip() if isinstance(col, tuple) else col.strip() for col in df.columns]
        print(f"Table {i} columns: {df.columns.tolist()}") # Print columns of each table
        if len(df.columns) == 4 and all(header in df.columns for header in expected_headers):
            return df
    raise ValueError("No suitable table found with 4 columns and expected headers.")


def q1_clean(df: pd.DataFrame) -> pd.DataFrame:
    """Clean columns: strip, UPPER Alpha-2/Alpha-3, cast Numeric to int (nullable), drop invalids."""

    # Ensure the expected columns exist before proceeding
    expected_cols = ["Country", "Alpha-2 code", "Alpha-3 code", "Numeric"]
    for col in expected_cols:
        if col not in df.columns:
            raise ValueError(f"Missing expected column: {col}")

    df["Numeric"] = pd.to_numeric(df["Numeric"], errors="coerce").astype("Int64")
    df["Alpha-2 code"] = df["Alpha-2 code"].str.strip().str.upper()
    df["Alpha-3 code"] = df["Alpha-3 code"].str.strip().str.upper()
    df["Country"] = df["Country"].str.strip()


    df = df.dropna(subset=["Country", "Numeric"])

    return df


def q1_sort_top(df: pd.DataFrame, top: int = 15) -> pd.DataFrame:
    """Sort descending by Numeric and return Top-N."""
    df = df.sort_values(by="Numeric", ascending=False)
    return df.head(top)

if __name__ == "__main__":
  url = "https://www.iban.com/country-codes"
  html = fetch_html(url)
  df_raw = q1_read_table(html)
  df_clean = q1_clean(df_raw)
  df_top15 = q1_sort_top(df_clean)

  df_clean.to_csv("data_q1.csv", index=False)

  print("Top 15 countries by numeric code:\n")
  print(df_top15.to_string(index=False))

Table 0 columns: ['Country', 'Alpha-2 code', 'Alpha-3 code', 'Numeric']
Top 15 countries by numeric code:

                                                   Country Alpha-2 code Alpha-3 code  Numeric
                                                    Zambia           ZM          ZMB      894
                                                     Yemen           YE          YEM      887
                                                     Samoa           WS          WSM      882
                                         Wallis and Futuna           WF          WLF      876
                        Venezuela (Bolivarian Republic of)           VE          VEN      862
                                                Uzbekistan           UZ          UZB      860
                                                   Uruguay           UY          URY      858
                                              Burkina Faso           BF          BFA      854
                                     Virgin Isl

/tmp/ipython-input-2986812657.py:6: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(html)


## Question 2 — Hacker News (front page)
**URL:** https://news.ycombinator.com/  
**Extract at least:** `rank`, `title`, `link`, `points`, `comments` (user optional)  
**Clean:** cast `points`/`comments`/`rank` → **int** (non-digits → 0), fill missing text fields  
**Output:** write **`data_q2.csv`** and **print a Top-15** sorted by `points` (desc, no charts)  
**Tip:** Each story is a `.athing` row; details (points/comments/user) are in the next `<tr>` with `.subtext`.


In [6]:
# --- Q2 Skeleton (fill the TODOs) ---
def q2_parse_items(html: str) -> pd.DataFrame:
    """Parse front page items into DataFrame columns:
       rank, title, link, points, comments, user (optional).
    TODO: implement with BeautifulSoup on '.athing' and its sibling '.subtext'.
    """
    raise NotImplementedError("TODO: implement q2_parse_items")

def q2_clean(df: pd.DataFrame) -> pd.DataFrame:
    """Clean numeric fields and fill missing values.
    TODO: cast points/comments/rank to int (non-digits -> 0). Fill text fields.
    """
    raise NotImplementedError("TODO: implement q2_clean")

def q2_sort_top(df: pd.DataFrame, top: int = 15) -> pd.DataFrame:
    """Sort by points desc and return Top-N. TODO: implement."""
    raise NotImplementedError("TODO: implement q2_sort_top")


In [16]:
# Q2 — Write your answer here


import requests
import pandas as pd
from bs4 import BeautifulSoup
import re

# Helper function to extract integers
def extract_int(text):
    if text:
        # Use regex to find digits
        match = re.search(r'\d+', str(text))
        if match:
            return int(match.group(0))
    return 0


def q2_parse_items(html: str) -> pd.DataFrame:
    """Parse front page items into DataFrame columns:
       rank, title, link, points, comments, user (optional).
    """
    soup = BeautifulSoup(html, 'html.parser')
    items = []

    for story in soup.select("tr.athing"):
        rank = story.select_one("span.rank")
        title_el = story.select_one("a.storylink")
        link = title_el.get('href') if title_el else ""
        title = title_el.get_text(strip=True) if title_el else ""

        subtext = story.find_next_sibling("tr").select_one("td.subtext")
        if subtext:
            points_el = subtext.select_one("span.score")
            points = points_el.get_text(strip=True) if points_el else ""
            user_el = subtext.select_one("a.hnuser")
            user = user_el.get_text(strip=True) if user_el else ""
            comments_el = subtext.find_all("a")[-1] if subtext.find_all("a") else None
            comments = comments_el.get_text(strip=True) if comments_el else ""
        else:
            points = user = comments = ""

        items.append({
            "rank": rank.get_text(strip=True) if rank else "",
            "title": title,
            "link": link,
            "points": points,
            "comments": comments,
            "user": user
        })

    return pd.DataFrame(items)

def q2_clean(df: pd.DataFrame) -> pd.DataFrame:
    """Clean numeric fields and fill missing values."""
    df = df.fillna("")

    df["rank"] = df["rank"].apply(extract_int)
    df["points"] = df["points"].apply(extract_int)
    df["comments"] = df["comments"].apply(extract_int)

    return df

def q2_sort_top(df: pd.DataFrame, top: int = 15) -> pd.DataFrame:
    """Sort by points desc and return Top-N."""
    df = df.sort_values(by="points", ascending=False)
    return df.head(top)

if __name__=="__main__":
  url = "https://news.ycombinator.com/"
  html = requests.get(url).text

  df_raw = q2_parse_items(html)
  df_clean = q2_clean(df_raw)
  df_top15 = q2_sort_top(df_clean)

  print("Top 15 Hacker News Stories by Points:\n")
  print(df_top15.to_string(index=False))

  df_clean.to_csv("data_q2.csv", index=False)

Top 15 Hacker News Stories by Points:

 rank title link  points  comments            user
   12                680       360    Projectiboga
    4                571       229       nekofneko
   16                529       163        vincvinc
    9                353        70  adam_gyroscope
    2                322       226          esnard
   25                295       422         hentrep
    1                278       136     tabletcorry
    6                231        89          costco
   18                223       109          nabla9
    8                181       102        glhaynes
   14                168       163             rzk
   15                140        67           kalli
   28                124        79         iagooar
   24                105        27 vitaly-pavlenko
   10                 97        17 stansApprentice
